# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

### Reading bigrams

In [1]:
import pandas as pd

In [2]:
bigrams = pd.read_csv("./data/bigrams.txt", sep="\t", encoding='latin-1', header=None)

In [3]:
bigrams.isna().sum()

0    0
1    4
2    3
dtype: int64

In [4]:
bigrams.dropna(inplace=True)

In [5]:
bigrams.head(5)

,0,1,2
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and


### Creating vocabulary dictionaries

In [6]:
bigram_dict = dict()
vocabulary = dict()

for index, row in bigrams.iterrows():
    key = (row[1], row[2])
    bigram_dict[key] = row[0]

    if (row[1] in vocabulary):
        vocabulary[row[1]] += 1
    else:
        vocabulary[row[1]] = 1

    if (row[2] in vocabulary):
        vocabulary[row[2]] += 1
    else:
        vocabulary[row[2]] = 1

### Main logic

In [7]:
!pip install editdistance -q


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [8]:
import editdistance

In [9]:
def best_editdistance(target_word, max_distance):
    # Initialize with target_word if it's in the vocabulary
    possible_matches = {target_word} if target_word in vocabulary else set()

    # Start searching at the next distance level
    distance = 1

    # Expand search until possible matches are found or max_distance is reached
    while not possible_matches and distance <= max_distance:
        for word in vocabulary:
            if editdistance.eval(target_word, word) == distance:
                possible_matches.add(word)
        distance += 1

    # Return the original word if no matches found within the edit distance
    return possible_matches if possible_matches else {target_word}

In [10]:
def best_unigram(target_word, edit_distance):
    # Find candidate words within the specified edit distance
    candidates = best_editdistance(target_word, edit_distance)
    
    # Calculate the unigram probability of each candidate
    total_vocabulary_size = len(vocabulary)
    suggestions = [(candidate, vocabulary[candidate] / total_vocabulary_size) 
                   for candidate in candidates if candidate in vocabulary]

    # Sort the suggestions based on probability, in descending order
    return sorted(suggestions, key=lambda suggestion: suggestion[1], reverse=True)

In [11]:
def best_bigram(bigram, edit_distance, left=True):
    # Find candidate words within the specified edit distance for the targeted part of the bigram
    target_word = bigram[0] if left else bigram[1]
    word_candidates = best_editdistance(target_word, edit_distance)

    # Generate candidate bigrams by pairing each candidate word with the fixed part of the original bigram
    candidate_bigrams = {(word, bigram[1]) if left else (bigram[0], word) for word in word_candidates}

    # Filter and rank candidate bigrams based on their probability in bigram_dict
    total_bigrams = len(bigram_dict)
    suggestions = [(c, bigram_dict[c] / total_bigrams) for c in candidate_bigrams if c in bigram_dict]

    # Return the suggestions sorted by probability in descending order
    return sorted(suggestions, key=lambda x: x[1], reverse=True)


In [12]:
def correct_sentence(sentence):
    words = sentence.lower().split()

    if not words:  # If the sentence is empty or only contains whitespace
        return ""

    corrected_sentence = []

    for i, word in enumerate(words):
        right_suggestions = []
        left_suggestions = []

        if i > 0:  # Look for suggestions based on the word to the left
            right_suggestions = best_bigram((words[i - 1], word), 1, left=False)
            right_suggestions = [(w, p) for ((_, w), p) in right_suggestions]

        if i < len(words) - 1:  # Look for suggestions based on the word to the right
            left_suggestions = best_bigram((word, words[i + 1]), 1, left=True)
            left_suggestions = [(w, p) for ((w, _), p) in left_suggestions]

        suggestions = left_suggestions + right_suggestions
        known_word = word in vocabulary

        # If no suggestions and the word is not known, try unigram suggestions
        if not suggestions and not known_word:
            suggestions = best_unigram(word, 1)

        # Combine probabilities for the same word across different suggestions
        probs = {}
        for w, p in suggestions:
            probs[w] = probs.get(w, 0) + p

        # Filter tiny probabilities, adjusting threshold based on known word
        threshold = 1e-5 if known_word else 1e-13
        probs = {w: p for w, p in probs.items() if p > threshold}

        # Choose the best suggestion or keep the original word
        corrected_word = max(probs, key=probs.get) if probs else word
        corrected_sentence.append(corrected_word)

    return " ".join(corrected_sentence)


In [13]:
def formatted_output_correction(sentence):
    print(f'Before: \t{sentence}')
    print(f'After:  \t{correct_sentence(sentence)}')
    print()

In [14]:
formatted_output_correction("dking sport")
formatted_output_correction("dking patient")
formatted_output_correction("he is able to cok great food")
formatted_output_correction("i like cokking crystal mth")
formatted_output_correction("i lke solvng mth prblems")
formatted_output_correction("She excels in playing the pano")

Before: 	dking sport
After:  	doing sport

Before: 	dking patient
After:  	dying patient

Before: 	he is able to cok great food
After:  	he is able to cook great food

Before: 	i like cokking crystal mth
After:  	i like cooking crystal meth

Before: 	i lke solvng mth prblems
After:  	i like solving math problems

Before: 	She excels in playing the pano
After:  	she excels in playing the piano



## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

- I am not an expert in NLP, so I decided to keep it simple.

- My approach uses bigrams, where word, which is taken into consideration about correction is looked with bigrams from both sides. It helps to grab twice more context (we are looking for 2 nearby words, not just 1). Also it helps to prevent situations where we have a chain of corrections after changing the first word.

- I use edit distance to find similar words for candidates and probability of unigram or bigram appearing in the vocabulary for the final choice.

- I use thresholds for replacing known and not known words, so correction would not always trigger. For known words the threshold is set for 1e-5 and for unknown for 1e-13. It is reasonable because we often do not want to change words, which are written correctly.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [15]:
w1 = 'hello'

In [16]:
import random

# Function to generate misspelled word
def generate_misspelled_word(word):
    choice = random.choice(["insert", "replace", "delete"])
    modified_word = list(word)

    if choice == "insert":
        # Insert a random symbol at a random position
        symbol = random.choice('qwertyuiopasdfghjklzxcvbnm')
        position = random.randint(0, len(modified_word))
        modified_word.insert(position, symbol)
    elif choice == "replace":
        # Replace a random character with a random symbol
        position = random.randint(0, len(modified_word) - 1)
        symbol = random.choice('qwertyuiopasdfghjklzxcvbnm')
        modified_word[position] = symbol
    elif choice == "delete":
        # Delete a random symbol
        if len(modified_word) > 1:
            position = random.randint(0, len(modified_word) - 1)
            del modified_word[position]

    return "".join(modified_word)

# Function to generate misspelled sentence
def generate_misspelled_sentence(sentence, num_words_to_modify=1):
    words = sentence.split()

    # Randomly select words to modify
    words_to_modify = random.sample(range(len(words)), min(num_words_to_modify, len(words)))

    # Generate misspelled version of the selected words
    for i in words_to_modify:
        words[i] = generate_misspelled_word(words[i])

    return " ".join(words)

In [17]:
def letters(input):
    return ''.join([c for c in input if c.isalpha() or c == ' '])

In [18]:
import re

# Function to read sentences from file
def read_sentences_from_file(filename):
    with open(filename, 'r') as file:
        sentences = file.read().split('\n')
        sentences = [re.sub(' +', ' ', letters(s).strip().lower()) for s in sentences]
    return sentences

In [19]:
sentences = read_sentences_from_file("./data/sentences.txt")

In [20]:
sentences[:5], len(sentences)

(['they fought a deadly war',
  'their house was farther away from my place',
  'so i think we would not be alive if our ancestors did not develop sciences and technologies',
  'imagine yourself you are working in factory just to do one thing like put air a on car if they fire you you will be destroyed because you do nt know more than to put air a in car',
  'for example they can play football whenever they want but the elders can not'],
 42)

In [21]:
mutated = [generate_misspelled_sentence(s, num_words_to_modify=2) for s in sentences]
mutated[:5]

['they foght y deadly war',
 'their holse was farther away ffom my place',
 's i think we would not be alive hif our ancestors did not develop sciences and technologies',
 'imagine yourself you are working in factory just po do one thing like put air a non car if they fire you you will be destroyed because you do nt know more than to put air a in car',
 'qfor example they can play football whenever they want but thf elders can not']

In [22]:
predictions = [correct_sentence(s) for s in sentences]
predictions[:5]

['they fought a deadly war',
 'their house was farther away from my place',
 'so i think we would not be alive if our ancestors did not develop sciences and technologies',
 'imagine yourself you are working in factory just to do one thing like put air a on car if they fire you you will be destroyed because you do nt know more than to put air a in car',
 'for example they can play football whenever they want but the elders can not']

In [23]:
total = 0
correct = 0

for s1, s2 in zip(sentences, predictions):
    total += 1
    if s1 == s2:
        correct += 1
    else:
        print('ERROR')
        print(f'Original sentence: \t {s1}')
        print()
        print(f'Prediction       : \t {s2}')
        print()

ERROR
Original sentence: 	 furthermore a tour guide will also provide safety and security for travel since they already know the dos and donts of the tour

Prediction       : 	 furthermore a tour guide will also provide safety and security for travel since they already know the dos and dots of the tour



The final accuracy is:

In [24]:
correct / total

0.9761904761904762